## Data Handling

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
np.random.seed(4444)

In [2]:
df = pd.read_csv('../../../../../../jaeyeun/01_nh_poc/16_re_label/split_70_15_15_re_3_prep_v10.csv')

df.head()

,file_name,raw_text,dataset_n,label_1,label_2,label_3,label_4,train_val_test,prep_v10_text
0,D1512065.txt,보 도 자 료\nhttp://www.motie.go.kr 2015년 12월28일(월...,0,0,0,0,0,0,가능N 국가N 기술N 표준원N 기술N 규제N 조정N 오광N 과장N 연구사N 기업N ...
1,R2002420.txt,보 도 자 료 작성과 공공데이터정책과\n2020년 2월 13일(목) 조간 (2. 1...,0,0,0,0,0,0,공공N 데이터N 정책N 이후N 남호N 공공N 데이터N 청년N 인턴N 참가N 청년N ...
2,R2002370.txt,2020년 2월 11일(화) 석간부터 보도하여 주시기 바랍니다. * 통신․방송․인터...,0,0,0,0,0,0,가능N 벤처N 투자N 과장N 벤처N 혁신N 정책N 과장N 벤처N 생태N 민간N 중심...
3,R2004373.txt,보도참고자료\nhttp://www.motie.go.kr 배포 즉시 보도하여 주시기 ...,0,0,0,0,0,0,즉시N 기정N 통부N 연구N 개발N 정책N 판식N 정보N 기술N 정책N 연구N 제도...
4,D1506066.txt,보 도 자 료\n보도일시 2015. 6. 12.(금) 조간(온라인 6. 11. 12...,0,0,0,0,0,0,온라인N 이후N 디지털N 정책N 과장N 지운N 스마트미디어N 엑스N 캠프N 신규N ...


* only for kdi dataset(exp1) & label_4

In [3]:
df = df[df['dataset_n'] == 0]

In [5]:
X_train_df = df[df['train_val_test'] == 0]
X_val_df = df[df['train_val_test'] == 1]
X_test_df = df[df['train_val_test'] == 2]
y_train = df[df['train_val_test'] == 0]['label_4']
y_val = df[df['train_val_test'] == 1]['label_4']
y_test = df[df['train_val_test'] == 2]['label_4']

In [6]:
print(X_train_df.shape)
print(X_val_df.shape)
print(X_test_df.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(7555, 9)
(1621, 9)
(1624, 9)
(7555,)
(1621,)
(1624,)


In [7]:
n_features = 100000
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.9, # 0.9 만큼의 문서 이상에서 나오면 거른다.
    min_df=5, # 5개 미만의 문서에서 나오면 거른다.
    sublinear_tf = True, # tf value를 완만하게 처리 (outlier 처리 효과)
    ngram_range = (1, 3),
    max_features=n_features)

* data is 'prep_v9_text'

In [8]:
X_train_tfidf = tfidf_vectorizer.fit(X_train_df['prep_v10_text'])
# X_test_tfidf = tfidf_vectorizer.fit_transform(prep_text_test)
# X_test_hash = hash_vectorizer.fit_transform(prep_text_test)

In [9]:
X_train = X_train_tfidf.transform(X_train_df['prep_v10_text'])
X_val = X_train_tfidf.transform(X_val_df['prep_v10_text'])
X_test = X_train_tfidf.transform(X_test_df['prep_v10_text'])

In [10]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(7555, 100000)
(1621, 100000)
(1624, 100000)


## Classify

* LightGBM RandomSearch

In [11]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgbm

In [13]:
lgbm_clf = lgbm.LGBMClassifier()

lgbm_param_grid = {
#     'gamma': [.05, .1, .3, .5, .7, .9, 1],

    # Core params
    'boosting': ['gbdt', 'goss'],
    'learning_rate': [.01, .015, .025, .05, .1, .125, .15],
    'num_leaves' : [5, 15, 30, 50, 100, 150, 200, 400], #it must be smaller than 2^(max_depth)
    'device_type' : ['gpu'],
    
    # Learning Control params
    'max_depth': [3, 5, 7, 9, 12, 15, 17, 25],
    'min_data_in_leaf': [10, 15, 20, 30, 50, 100],
    'min_sum_hessian_in_leaf': [.0001, .0005,.001, .002, .005, .01],
    'bagging_fraction': np.linspace(0.6, 1, 5), # aliases subsample
    'bagging_freq': range(0,10),
    'feature_fraction': np.linspace(0.6, 1, 5),

    # Dataset params
    'max_bin': [16, 32, 64, 128, 256]
}

fit_params={
    'early_stopping_rounds' : 30,
    'eval_metric' : "multi_error",
    'eval_set' : [(X_train, y_train), (X_val, y_val)]
}


# Create a random search object
lgbm_random = RandomizedSearchCV(estimator = lgbm_clf,
                                param_distributions = lgbm_param_grid,
                                n_iter = 20, # n_iters in param combinations
                                scoring='accuracy',
                                n_jobs=-1,
                                cv = 5,
                                refit=True,
                                return_train_score = True,
                                verbose=10)

# Fit to the training data
lgbm_random.fit(X_train, y_train, **fit_params)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   43.8s
[Parallel(n_jobs=-1)]: Done  48 out of 100 | elapsed:   50.2s remaining:   54.3s
[Parallel(n_jobs=-1)]: Done  59 out of 100 | elapsed:  4.7min remaining:  3.3min
[Parallel(n_jobs=-1)]: Done  70 out of 100 | elapsed:  7.6min remaining:  3.3min
[Parallel(n_jobs=-1)]: Done  81 out of 100 | elapsed:  8.2min remaining:  1.9min
[Parallel(n_jobs=-1)]: Done  92 out of 100 | elapsed: 12.4min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 36.8min finished


[1]	training's multi_error: 0.235738	training's multi_logloss: 1.82131	valid_1's multi_error: 0.319556	valid_1's multi_logloss: 1.87513
Training until validation scores don't improve for 30 rounds
[2]	training's multi_error: 0.111714	training's multi_logloss: 1.55344	valid_1's multi_error: 0.206046	valid_1's multi_logloss: 1.64136
[3]	training's multi_error: 0.078094	training's multi_logloss: 1.35463	valid_1's multi_error: 0.172733	valid_1's multi_logloss: 1.46759
[4]	training's multi_error: 0.0622105	training's multi_logloss: 1.19538	valid_1's multi_error: 0.163479	valid_1's multi_logloss: 1.33374
[5]	training's multi_error: 0.052548	training's multi_logloss: 1.06371	valid_1's multi_error: 0.161629	valid_1's multi_logloss: 1.22243
[6]	training's multi_error: 0.0455328	training's multi_logloss: 0.952589	valid_1's multi_error: 0.156076	valid_1's multi_logloss: 1.12906
[7]	training's multi_error: 0.0418266	training's multi_logloss: 0.857047	valid_1's multi_error: 0.154226	valid_1's multi

RandomizedSearchCV(cv=5, estimator=LGBMClassifier(), n_iter=20, n_jobs=-1,
                   param_distributions={'bagging_fraction': array([0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'bagging_freq': range(0, 10),
                                        'boosting': ['gbdt', 'goss'],
                                        'device_type': ['gpu'],
                                        'feature_fraction': array([0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'learning_rate': [0.01, 0.015, 0.025,
                                                          0.05, 0.1, 0.125,
                                                          0.15],
                                        'max_bin': [16, 32, 64, 128, 256],
                                        'max_depth': [3, 5, 7, 9, 12, 15, 17,
                                                      25],
                                        'min_data_in_leaf': [10, 15, 20, 30, 50,
                     

In [14]:
pred_train = lgbm_random.best_estimator_.predict(X_train)
pred_val = lgbm_random.best_estimator_.predict(X_val)
pred_test = lgbm_random.best_estimator_.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))
print(accuracy_score(y_test, pred_test))

0.999602911978822
0.8729179518815546
0.8793103448275862


In [21]:
cv_result_df = pd.DataFrame(lgbm_random.cv_results_)

df_list = []
for i in range(20):
    df_list.append(pd.DataFrame([cv_result_df.loc[i, "params"]]))
    
param_table = pd.concat(df_list)

In [22]:
param_table

,num_leaves,min_sum_hessian_in_leaf,min_data_in_leaf,max_depth,max_bin,learning_rate,feature_fraction,device_type,boosting,bagging_freq,bagging_fraction
0,150,0.0020,15,7,256,0.050,1.0,gpu,goss,6,0.7
0,30,0.0005,20,5,32,0.150,0.7,gpu,goss,7,0.9
0,400,0.0100,20,25,128,0.125,0.9,gpu,gbdt,3,1.0
0,15,0.0020,20,17,16,0.015,0.9,gpu,gbdt,1,0.7
0,100,0.0001,10,25,16,0.050,0.9,gpu,goss,2,0.7
0,200,0.0050,100,12,256,0.015,0.8,gpu,goss,9,1.0
0,15,0.0100,30,3,16,0.150,1.0,gpu,goss,7,0.9
0,400,0.0100,100,17,256,0.125,0.8,gpu,goss,9,0.6
0,100,0.0050,10,3,32,0.150,1.0,gpu,goss,2,0.9
0,150,0.0010,30,9,256,0.015,0.8,gpu,goss,1,0.6


In [24]:
param_table['mean_test_score'] = cv_result_df['mean_test_score'].values

param_table.sort_values(by='mean_test_score', axis=0)

,num_leaves,min_sum_hessian_in_leaf,min_data_in_leaf,max_depth,max_bin,learning_rate,feature_fraction,device_type,boosting,bagging_freq,bagging_fraction,mean_test_score
0,200,0.0050,100,12,256,0.015,0.8,gpu,goss,9,1.0,0.808339
0,15,0.0020,20,17,16,0.015,0.9,gpu,gbdt,1,0.7,0.849239
0,200,0.0001,20,3,64,0.100,0.9,gpu,goss,4,1.0,0.864196
0,100,0.0001,100,15,16,0.100,0.7,gpu,gbdt,3,0.7,0.866314
0,50,0.0001,100,9,16,0.125,1.0,gpu,gbdt,0,0.7,0.868432
0,400,0.0100,20,25,128,0.125,0.9,gpu,gbdt,3,1.0,0.871079
0,150,0.0020,15,7,256,0.050,1.0,gpu,goss,6,0.7,NaN
0,30,0.0005,20,5,32,0.150,0.7,gpu,goss,7,0.9,NaN
0,100,0.0001,10,25,16,0.050,0.9,gpu,goss,2,0.7,NaN
0,15,0.0100,30,3,16,0.150,1.0,gpu,goss,7,0.9,NaN


In [28]:
lgbm_random.cv_results_

{'mean_fit_time': array([  22.59727244,   12.46462908, 2136.06647286,  494.18496127,
          25.01536522,  450.9778533 ,    8.82484307,    8.78429046,
          21.14750423,   13.36977019,   18.56929679,   12.25911927,
         153.38956347,    5.69440131,  234.79789023,  451.56298347,
         293.94690819,  407.76079068,  211.82742634,  113.81169605]),
 'std_fit_time': array([  1.0042422 ,   0.44042046,  61.91160834,  27.45322547,
          0.56140383,  13.29536337,   1.24706918,   0.95807545,
          0.6341597 ,   3.02164357,   0.5441458 ,   0.39297257,
        104.59293803,   1.05847347,   2.65094173, 352.38311159,
        104.72388129, 206.52283745,  24.24966777,  14.53260769]),
 'mean_score_time': array([ 0.        ,  0.        ,  0.332477  ,  0.58498068,  0.        ,
         1.32518663,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.21558771, 11.11133976,  0.22855997,  0.1498105 ,  0.23